In [1]:
from google.cloud import bigquery
from google.cloud import storage
#https://ruddra.com/tfrecords-pandas/

In [10]:
client_bq = bigquery.Client()

sql = """

 SELECT  company,count_detail
 FROM `pongthorn.SMartDW.incident` 
 WHERE DATE(open_datetime) > "2023-02-01" order by updated_at desc LIMIT 10
"""

query_result=client_bq.query(sql)
df=query_result.to_dataframe()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company       10 non-null     object
 1   count_detail  10 non-null     Int64 
dtypes: Int64(1), object(1)
memory usage: 298.0+ bytes


,company,count_detail
0,INET-(NetApp and Non HPE),3
1,INET-(NetApp and Non HPE),3
2,KTBCS,4
3,KTBCS,14
4,KTBCS,2


In [13]:
import tensorflow as tf

def create_tf_records_example(features):
    tf_example = tf.train.Example(
        features=tf.train.Features(feature={
            'company': tf.train.Feature(bytes_list=tf.train.BytesList(
                value=[features[0].encode('utf-8')])),
            'count_detail': tf.train.Feature(float_list=tf.train.FeatureLists(
                value=[features[1]])),
    }))
    return tf_example

In [14]:
writer = tf.io.TFRecordWriter("incident_tfrecords.tfrecords")
for row in df.values:
    # print(row)
    example = create_tf_records_example(row)
    writer.write(example.SerializeToString())

ValueError: Protocol message FeatureLists has no "value" field.